In [179]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import importlib
import connect4
importlib.reload(connect4);
from connect4 import Connect4

In [180]:
#
# Create the model and optimizer
# 
class Connect4Cnn(nn.Module):
    def __init__(self):
        super(Connect4Cnn, self).__init__()
        self.conv1 = nn.Conv2d(3, 64,  kernel_size=3, padding=1)    
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        #self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.out = nn.Linear(128*6*7, 7)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        #x = F.relu(self.conv3(x))
        x = x.view(-1)
        x = self.out(x)
        return x

model = Connect4Cnn()
optimizer = torch.optim.RMSprop(model.parameters(), lr = 0.01, alpha = 0.99)
sum(p.numel() for p in model.parameters() if p.requires_grad)

113287

In [181]:
#
# Load model from checkpoint
#
games = 0
if games > 0:
    cp = torch.load(f'connect4-{games}.nn');
    model.load_state_dict(cp['model_state_dict']);
    optimizer.load_state_dict(cp['optimizer_state_dict']);

In [182]:
@torch.no_grad()
def checkpoint(step):
    train = model.training
    if train: model.eval();
    print(f"{step}: checkpoint...")
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, f'connect4-{step}.nn');

    dummy_input = Connect4().state
    torch.onnx.export(model, dummy_input, f"connect4-{step}.onnx");

    print(f"{step}: checkpoint saved.")
    if train: model.train();


In [183]:
#
# Validation
#
@torch.no_grad()
def validate(numberOfGames):
    print('Validation...')
    xw, xd, xl, ow, od, ol  = connect4.validate(model, numberOfGames)
    xtotal = xw + xd + xl
    ototal = ow + od + ol
    print(f"Cross: {100*xw/xtotal:.2f}% of {xtotal} ({xw}/{xd}/{xl})")
    print(f"Circle: {100*(ow+od)/ototal:.2f}% of {ototal} ({ow}/{od}/{ol})")

In [184]:
#
# TRAINING
#

epsilon = 0.3
gamma = 0.9

log_interval = 5000
validation_interval = 50000
validation_games = 10000
checkpoint_interval = 50000
losses = []
model.train()
for _ in range(100000):
    env = Connect4()
    done = False
    games += 1
    moves = 0

    qstack = []

    while env.winner == 0 and not env.full:
        q = model(env.state)
        targetq = q.detach().clone()
        validmoves = [a for a in range(7) if env.is_valid(a)]

        e = random.uniform(0, 1)
        if e < epsilon:
            action = random.choice(validmoves)
        else:
            action = max(validmoves, key=lambda x: q[x])
        
        qstack.append((q, action, validmoves))
        env.move(action)
        moves += 1

    qlist = []
    targetlist = []

    (q, action, validmoves) = qstack.pop()
    targetq = q.clone().detach()
    targetq[action] = 1 if env.winner != 0 else 0
    qlist.append(q)
    targetlist.append(targetq)

    while len(qstack) > 0:
        next_q = targetq
        nextvalidmoves = validmoves
        (q, action, validmoves) = qstack.pop()
        targetq = q.clone().detach()
        next_max = -max([next_q[a] for a in nextvalidmoves]).item()
        targetq[action] = -0.1 + gamma * next_max
        qlist.append(q)
        targetlist.append(targetq)

    qtensor = torch.stack(qlist)
    targettensor = torch.stack(targetlist)
    loss = F.mse_loss(qtensor, targettensor,)
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if games % log_interval == 0:
        print(f'{games}: average loss: {sum(losses)/len(losses)}')
        losses = []
    if games % validation_interval == 0:
        validate(validation_games)
    if games % checkpoint_interval == 0:
        checkpoint(games)


5000: average loss: 4.723663315574685


### Connect4 CNN training 

#### Layers: C64-C128
&gamma;: 0.9, &epsilon;: 0.3
optimizer: RMSprop lr: 0.01 &alpha;: = 0.999

| Games     | &gamma; | &epsilon; | Loss                  | Cross     | Circle | Remarks
| :-------: | :---:   | :---:     | :-----------:         | :-------: | :-------: | :-------:
|  50000    | 0.9     | 0.3       ||
| 100000    | 0.9     | 0.3       ||


#### Layers: C64-C128-C128
&lr;: 0.1, &gamma;: 0.9, &epsilon;: 0.5

| Games     | Loss                  | Cross     | Circle | Remarks
| :-------: | :-----------:         | :-------: | :-------: | :-------:
|  50000    | 0.005292380097397108  | 89.98%    | 84.68%
| 100000    | 0.004793603768121102  | 91.16%    | 85.10%
| 150000    | 0.004057415647386188  | 86.90%    | 88.80%
| 200000    | 0.0034653012876227876 | 95.20%    | 90.72%
| 250000    | 0.003226012402748529  | 95.66%    | 93.82% 
| 300000    | 0.00299808547905468   | 94.62%    | 94.32%
| 350000    | 0.0028812737085465415 | 96.62%    | 96.18%
| 400000    | 0.0025629888497351204 | 97.60%    | 94.76% 
| 450000    | 0.002434644917307014  | 97.96%    | 96.40%
| 500000    | 0.002341157681126242  | 98.20%    | 94.50% | optimizing training loop
| 550000    | 0.0022111331460142537 | 97.54%    | 95.18%
| 600000    | 0.0022098078040602557 | 97.46%    | 96.14% | reducing &epsilon; to 0.3
| 650000    | 0.0017878218773417303 | 97.74%    | 96.46% |
| 700000    | 0.0017158431361114707 | 96.78%    | 97.42% | refactoring learning loop
| 750000    | 0.0022741290108140673 | 97.44%    | 97.00% |
| 800000    | 0.0024079284169672066 | 96.68%    | 96.14% | consolidate loss, &alpha;: 0.05, &epsilon;: 0.2, &epsilon;': 0.8
| 850000    | 0.0012254708257347828 | 96.12%    | 96.12% |
| 900000    | 0.0010383383095133696 | 95.60%    | 97.16% | &alpha;: 0.025, &epsilon;: 0.2
| 950000    | 0.0020861899321079134 | 96.88%    | 97.22% |
| 1000000   | 0.0022770740728620693 | 96.64%    | 97.14% |
